In [2]:
!pip install emoji

     |████████████████████████████████| 133kB 7.6MB/s 


### **Imporing** **Libraries** 

In [3]:

import numpy as np
import pandas as pd
import emoji
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import LSTM,Dropout,Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical



In [4]:
emoji.EMOJI_UNICODE

{'en': {':1st_place_medal:': '🥇',
  ':2nd_place_medal:': '🥈',
  ':3rd_place_medal:': '🥉',
  ':AB_button_(blood_type):': '🆎',
  ':ATM_sign:': '🏧',
  ':A_button_(blood_type):': '🅰',
  ':Afghanistan:': '🇦🇫',
  ':Albania:': '🇦🇱',
  ':Algeria:': '🇩🇿',
  ':American_Samoa:': '🇦🇸',
  ':Andorra:': '🇦🇩',
  ':Angola:': '🇦🇴',
  ':Anguilla:': '🇦🇮',
  ':Antarctica:': '🇦🇶',
  ':Antigua_&_Barbuda:': '🇦🇬',
  ':Aquarius:': '♒',
  ':Argentina:': '🇦🇷',
  ':Aries:': '♈',
  ':Armenia:': '🇦🇲',
  ':Aruba:': '🇦🇼',
  ':Ascension_Island:': '🇦🇨',
  ':Australia:': '🇦🇺',
  ':Austria:': '🇦🇹',
  ':Azerbaijan:': '🇦🇿',
  ':BACK_arrow:': '🔙',
  ':B_button_(blood_type):': '🅱',
  ':Bahamas:': '🇧🇸',
  ':Bahrain:': '🇧🇭',
  ':Bangladesh:': '🇧🇩',
  ':Barbados:': '🇧🇧',
  ':Belarus:': '🇧🇾',
  ':Belgium:': '🇧🇪',
  ':Belize:': '🇧🇿',
  ':Benin:': '🇧🇯',
  ':Bermuda:': '🇧🇲',
  ':Bhutan:': '🇧🇹',
  ':Bolivia:': '🇧🇴',
  ':Bosnia_&_Herzegovina:': '🇧🇦',
  ':Botswana:': '🇧🇼',
  ':Bouvet_Island:': '🇧🇻',
  ':Brazil:': '🇧🇷',
  ':British_Indi

In [5]:
dictionary = { "0":":heart_suit:",
               "1":":baseball:",
               "2":":grinning_face_with_big_eyes:",
                "3":":disappointed_face:",
                "4":":fork_and_knife:",
                "5":":hundred_points:",
                "6":":fire:",
                "7":":face_blowing_a_kiss:",
                "8":":chestnut:",
                "9":":flexed_biceps:",
                "10":":open_hands:"
               


    
    
            }

In [6]:
for emoji_var in dictionary.values():
  print(emoji.emojize(emoji_var))

♥
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪
👐


### **Playing with data**

In [7]:
#Change the path, it was content/data/file name when I did it in colab
training_data = pd.read_csv('/content/data/train_emoji.csv',header= None,error_bad_lines=False)
test_data = pd.read_csv('/content/data/test_emoji.csv',header = None,error_bad_lines=False)

FileNotFoundError: ignored

In [ ]:
#Let's check our dataframe
training_data.head()

In [ ]:
data = training_data.values

In [ ]:
X_train = training_data[0]
Y_train = training_data[1]

X_test = test_data[0]
Y_test = test_data[1]

In [ ]:
# Let's have a look at few sentence and their corresponding emoji.

for i in range(100):
  txt = X_train[i]
  corr_emoji = emoji.emojize(dictionary[str(Y_train[i])])
  print(txt,corr_emoji)

###  Creating word to embeddings dictionary 

In [ ]:
Embedding_file = open('/content/glove.6B.50d.txt',encoding = 'utf-8')

In [ ]:
word_2_embedding = {}
for line in Embedding_file:
    values = line.split()
    word =  values[0]
    embedding = np.asarray(values[1:],dtype= 'float')
    word_2_embedding[word] = embedding
Embedding_file.close()

### Converting sentence to vectors

In [ ]:
def embedding_output(X):
    max_len = 10 #Max length of sentence
    embd_dim = 50
    embedding_mat = np.zeros((X.shape[0],max_len,embd_dim))
    
    # Getting embedding of every word in every sentence
    for i in range(X.shape[0]):        
        X[i] = X[i].split()
        

        for j in range(len(X[i])):
            try:
                 embedding_mat[i][j] = word_2_embedding[X[i][j].lower()]
            except:
                 embedding_mat[i][j] = np.zeros((50,))
    return embedding_mat

In [ ]:
print(X_train[0])
print(len(X_train[0]))

In [ ]:
embedding_mat_train = embedding_output(X_train)
embedding_mat_test = embedding_output(X_test)

In [ ]:
print(embedding_mat_train.shape)
print(embedding_mat_test.shape)

In [ ]:
Y_train = to_categorical(Y_train,num_classes=5)
Y_test = to_categorical(Y_test,num_classes=5)

### **Applying deep learning 😁**

In [ ]:
#Defining RNN and LSTM models
model = Sequential()
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer= 'adam',metrics=['accuracy'])
model.summary()

In [ ]:
hist = model.fit(embedding_mat_train,Y_train,epochs = 100,batch_size = 64,shuffle=True,validation_split=0.2)

In [ ]:
predictions = model.predict_classes(embedding_mat_test)

In [ ]:
print(predictions)

In [ ]:
model.evaluate(embedding_mat_test,Y_test)

In [ ]:
early_stop =  EarlyStopping(monitor='val-acc',patience = 10)
check_point = ModelCheckpoint("best_model.h5",monitor='val-loss ',verbose = True,save_best_only=True)

In [ ]:
hist = model.fit(embedding_mat_train,Y_train,epochs = 100,batch_size = 64,shuffle=True,validation_split=0.2)

In [ ]:
# model.load_weights("best_model.h5")
model.evaluate(embedding_mat_test,Y_test)